# 0. Initialisation


In [ ]:
import numpy as np

import scipy.stats
from scipy.stats import norm
from scipy.stats import t


from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

import matplotlib.pyplot as plt
import statistics
import pandas as pd
import time

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping

from mpl_toolkits.mplot3d import Axes3D

# 1. Functions

## Cholesky Decomposition

In [ ]:
def cholesky_cov(volatilities, correlation_matrix):

    correlation_matrix = np.atleast_2d(correlation_matrix)
    D = np.diag(volatilities)
    Sigma = D @ correlation_matrix @ D
    return np.linalg.cholesky(Sigma)

## Simulating Asset Paths Function

In [ ]:
def simulate_asset_paths(n_simulations, n_steps, n_assets, time_to_maturity, risk_free_rate, volatilities, correlation_matrix, cholesky_cov, initial_price):

    dt = time_to_maturity / n_steps
    paths = np.zeros((n_simulations, n_steps + 1, n_assets))
    paths[:, 0, :] = initial_price
    L = cholesky_cov(volatilities, correlation_matrix)

    for i in range(n_steps):
        z = np.random.standard_normal((n_simulations, n_assets))
        increments = (z @ L.T) * np.sqrt(dt)
        drift = (risk_free_rate - 0.5 * volatilities**2) * dt
        paths[:, i + 1, :] = paths[:, i, :] * np.exp(drift + increments)
    return paths

## Black Scholes

In [ ]:
# European Call
def black_scholes_call_price(S0, K, r, T, sigma):
    d1 = (np.log(S0 / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S0 * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

# European Put
def black_scholes_put_price(S0, K, r, T, sigma):
    d1 = (np.log(S0 / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return -S0 * norm.cdf(-d1) + K * np.exp(-r * T) * norm.cdf(-d2)

## Longstaff-Schwartz Algorithm

In [ ]:
def longstaff_schwartz_pricing(
    asset_paths,
    strike_price,
    risk_free_rate,
    time_to_maturity,
    exercise_points,
    func_payoff,
    func_price,
    func_in_the_money,
    basis_degree = 3
    ):
    # Dummy functions func_payoff, func_price and func_in_the_money wher created to make each algorithm generic

    n_simulations, n_steps_plus_one, n_assets = asset_paths.shape
    dt = time_to_maturity / (n_steps_plus_one - 1)

    option_value = func_payoff(asset_paths[:, -1, :], strike_price)   # Initialise option value at maturity, asset_paths[asset number, time step, simulation]

    # Iterate backward through exercise points
    for t in reversed(exercise_points):
        current_asset_prices = asset_paths[:, t, :]
        in_the_money = func_in_the_money(current_asset_prices, strike_price)

        if len(in_the_money) > 0: # Necessary for analysing boundary conditions
            if n_assets == 1:
                 x = current_asset_prices[in_the_money, 0]
            else:
                # Pass strike_price to func_price if it's strangle_spread_value
                if func_price.__name__ == 'strangle_spread_value':
                    x = func_price(current_asset_prices, in_the_money, strike_price)
                else:
                    x = func_price(current_asset_prices, in_the_money)
            y = option_value[in_the_money] * np.exp(-risk_free_rate * dt)

            # Least squares regression using basis function
            regression_coeffs = np.polyfit(x, y, basis_degree)
            continuation_value = np.polyval(regression_coeffs, x)

            # Determine exercise points
            exercise_value = func_payoff(current_asset_prices[in_the_money, :], strike_price)
            relative_exercise_indices = np.where(exercise_value > continuation_value)[0]
            exercise_indices = in_the_money[relative_exercise_indices]

            # Update option value for exercising paths only
            option_value[exercise_indices] = exercise_value[relative_exercise_indices]

            # Non-exercising paths discount future value
            not_exercising_indices = np.setdiff1d(in_the_money, exercise_indices)
            option_value[not_exercising_indices] *= np.exp(-risk_free_rate * dt)

        # Paths not in-the-money just get discount
        not_in_the_money = np.setdiff1d(np.arange(n_simulations), in_the_money)
        option_value[not_in_the_money] *= np.exp(-risk_free_rate * dt)

    return option_value

## KKT Algorithm

In [ ]:
def KKT_pricing_with_validation(
    asset_paths,
    strike_price,
    risk_free_rate,
    time_to_maturity,
    exercise_points,
    func_payoff,
    func_price,
    func_in_the_money,
    # Validation and nn parameters
    nn_epochs = 500,
    validation_split = 0.2,
    early_stopping_patience = 30,
    plot_validation = False
):

    n_simulations, n_steps_plus_one, n_assets = asset_paths.shape
    dt = time_to_maturity / (n_steps_plus_one - 1)

    # Initialize option value at maturity
    option_value = func_payoff(asset_paths[:, -1, :], strike_price)

    # Define the model ONCE
    model = keras.Sequential([
        keras.layers.Dense(32, activation='relu', input_shape=(1,)),
        keras.layers.Dense(32, activation='relu'),
        keras.layers.Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')

    histories = {}

    # Determine actual validation split based on number of simulations
    current_validation_split = validation_split if n_simulations >= 4 else 0 # Use 0 validation split if less than 4 simulations

    # Iterate backward through exercise points
    for t in reversed(exercise_points):
        current_asset_prices = asset_paths[:, t, :] # plane t of the prism
        in_the_money = func_in_the_money(current_asset_prices, strike_price)

        if len(in_the_money) > 0:
            # Prepare data for the current step
            if func_price.__name__ == 'strangle_spread_value':
                x = func_price(current_asset_prices, in_the_money, strike_price)
            else:
                x = func_price(current_asset_prices, in_the_money)
            y = option_value[in_the_money] * np.exp(-risk_free_rate * dt)

            # Format the 1D feature array for keras
            if x.ndim == 1:
                x = x.reshape(-1, 1)

            # Define the EarlyStopping callback
            # Only use early stopping if validation split is active and sufficient data
            callbacks = [EarlyStopping(
                monitor='val_loss',
                patience=early_stopping_patience,
                restore_best_weights=True,
                verbose=0
            )] if current_validation_split > 0 and len(x) > 1 else [] # Removed callbacks if insufficient data for validation

            # Train the EXISTING model starting with the weights from step t+1
            history = model.fit(
                x, y,
                epochs=nn_epochs,
                validation_split=current_validation_split,
                callbacks=callbacks,
                verbose=0
            )

            # Save history for this step if validation was used
            if current_validation_split > 0 and history.history:
                histories[t] = history.history


            # Predict continuation values using the fine-tuned model
            continuation_value = model.predict(x, verbose=0).flatten()

            # specifying option value based on whether ITM & exercised or not
            exercise_value = func_payoff(current_asset_prices[in_the_money, :], strike_price)
            relative_exercise_indices = np.where(exercise_value > continuation_value)[0]
            exercise_indices = in_the_money[relative_exercise_indices]
            option_value[exercise_indices] = exercise_value[relative_exercise_indices]
            not_exercising_indices = np.setdiff1d(in_the_money, exercise_indices)
            option_value[not_exercising_indices] *= np.exp(-risk_free_rate * dt)

        # For paths not in the money, discount as before
        not_in_the_money = np.setdiff1d(np.arange(n_simulations), in_the_money)
        option_value[not_in_the_money] *= np.exp(-risk_free_rate * dt)

    # Final price calculation
    final_price = np.mean(option_value)
    print(f"\nTraining complete. Best Lower Bound (from validation): {final_price:.4f}")

    # Plot validation if requested and validation data was used
    if plot_validation and histories:
      first_t = min(histories.keys())
      last_t = max(histories.keys())

      fig, axes = plt.subplots(1, 2, figsize=(14,5), sharey=True)

      # First time point
      axes[0].plot(histories[first_t]['loss'], label='Train loss')
      if 'val_loss' in histories[first_t]:
          axes[0].plot(histories[first_t]['val_loss'], label='Val loss', linestyle='--')
      axes[0].set_title(f"Validation at exercise point t={first_t}")
      axes[0].set_xlabel("Epochs")
      axes[0].set_ylabel("MSE Loss")
      axes[0].legend()

      # Last time point
      axes[1].plot(histories[last_t]['loss'], label='Train loss')
      if 'val_loss' in histories[last_t]:
          axes[1].plot(histories[last_t]['val_loss'], label='Val loss', linestyle='--')
      axes[1].set_title(f"Validation at exercise point t={last_t}")
      axes[1].set_xlabel("Epochs")
      axes[1].legend()

      plt.suptitle("Training vs Validation Loss at Start and End Exercise Points")
      plt.tight_layout()
      plt.show()

    return final_price

## Deep Dual Method

In [ ]:
# Defining the deep dual pricing algorithm function with validation
def deep_dual_pricing(
    asset_paths,
    strike_price,
    risk_free_rate,
    time_to_maturity,
    exercise_points, # This parameter isnt used in the function; it is there for consistency with KKT
    func_payoff,
    num_epochs=500, # Increased default epochs as early stopping will handle it
    learning_rate=0.001,
    hidden_dim=64,
    # below for validation
    validation_split=0.2, # Fraction of data to use for validation
    early_stopping_patience= 80, # Stop if val_loss doesn't improve for x epochs
    plot_validation = False
):

    # split the data into a training and validation set
    num_sims_total = asset_paths.shape[0]
    num_val = int(num_sims_total * validation_split)
    num_train = num_sims_total - num_val

    train_paths = asset_paths[:num_train, :, :]
    val_paths = asset_paths[num_train:, :, :]

    # Convert to TensorFlow constants
    train_paths_tensor = tf.constant(train_paths, dtype=tf.float32)
    val_paths_tensor = tf.constant(val_paths, dtype=tf.float32)

    _, num_timesteps, num_assets = train_paths_tensor.shape

    # making the discount factors to discount payoffs with
    dt = time_to_maturity / (num_timesteps - 1)
    time_grid = tf.linspace(0.0, time_to_maturity, num_timesteps)
    discount_factors = tf.exp(-risk_free_rate * time_grid)

    # function to calculate payoff at each time and then disocunt to t=0
    def calculate_discounted_payoffs(paths_tensor):
        payoffs_list = []
        for i in range(num_timesteps):
            payoff_at_t = func_payoff(paths_tensor[:, i, :], strike_price)
            payoffs_list.append(payoff_at_t * discount_factors[i])
        return tf.stack(payoffs_list, axis=1)

    train_discounted_payoffs = calculate_discounted_payoffs(train_paths_tensor)
    val_discounted_payoffs = calculate_discounted_payoffs(val_paths_tensor)

    # Defining the neural network function
    def create_martingale_control_net(input_dim, output_dim, hidden_dim=64):
        """Creates a Keras Sequential model."""
        model = keras.Sequential([
            layers.Input(shape=(input_dim,)),
            layers.Dense(hidden_dim, activation='relu'),
            layers.Dense(hidden_dim, activation='relu'),
            #layers.Dense(hidden_dim, activation='relu'),
            #layers.Dense(hidden_dim, activation='relu'),
            layers.Dense(output_dim)
        ])
        return model

    # initialising
    input_dim = num_assets + 1
    output_dim = num_assets
    model = create_martingale_control_net(input_dim, output_dim, hidden_dim)
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

    # training loop and related functions (with validation)

    # initialising variables for early stopping
    best_val_loss = float('inf')
    patience_counter = 0

    # Lists to store loss history for plotting
    train_loss_history = []
    val_loss_history = []

    # function to get the upper bound (will call in the epoch loop)
    def calculate_upper_bound(paths_tensor, discounted_payoffs):
        num_sims = paths_tensor.shape[0]
        dM_list = []
        for i in range(num_timesteps - 1):
            current_S = paths_tensor[:, i, :]
            current_t = tf.broadcast_to(time_grid[i], [num_sims, 1])
            net_input = tf.concat([current_t, current_S], axis=1)
            Z_t = model(net_input)
            next_S = paths_tensor[:, i + 1, :]
            expected_next_S = current_S * tf.exp(risk_free_rate * dt)
            unexpected_move = next_S - expected_next_S
            dM = tf.reduce_sum(Z_t * unexpected_move, axis=1)
            dM_list.append(dM)

        dM_stacked = tf.stack(dM_list, axis=1)
        M_t = tf.concat([tf.zeros((num_sims, 1)), tf.cumsum(dM_stacked, axis=1)], axis=1)
        adjusted_payoff = discounted_payoffs - M_t
        pathwise_max = tf.reduce_max(adjusted_payoff, axis=1)
        return tf.reduce_mean(pathwise_max)

    for epoch in range(num_epochs):
        # training step
        with tf.GradientTape() as tape:
            # Loss is calculated ONLY on the training data
            train_loss = calculate_upper_bound(train_paths_tensor, train_discounted_payoffs)

        gradients = tape.gradient(train_loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables)) # updating the model

        # validation step
        # loss is calculated on validation data with no gradient tracking
        val_loss = calculate_upper_bound(val_paths_tensor, val_discounted_payoffs)
        val_loss_numpy = val_loss.numpy()

        # Store loss history
        train_loss_history.append(train_loss.numpy())
        val_loss_history.append(val_loss_numpy)

        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss.numpy():.4f}, Val Loss: {val_loss_numpy:.4f}')

        # early stopping logic
        if val_loss_numpy < best_val_loss:
            best_val_loss = val_loss_numpy
            patience_counter = 0 # Resetting patience because we found a better model
        else:
            patience_counter += 1 # Increment patience because performance did not improve this epoch

        if patience_counter >= early_stopping_patience:
            break

    # The final reported price is the BEST validation loss found during training

    if plot_validation:
        plt.figure(figsize=(10, 6))
        plt.plot(train_loss_history, label='Train Loss')
        plt.plot(val_loss_history, label='Val Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        plt.grid(True)
        plt.show()


    return float(best_val_loss)

## Option Payoffs

### Basket Option

In [ ]:
def basket_call_option_payoff(asset_prices, strike_price):
    return np.maximum(np.sum(asset_prices, axis=1) - strike_price, 0)

def basket_value(current_asset_prices, in_the_money):
    return np.sum(current_asset_prices[in_the_money, :], axis=1)

def basket_in_the_money(current_asset_prices, strike_price):
    return np.where(np.sum(current_asset_prices, axis=1) > strike_price)[0]

### Rainbow Option

In [ ]:
def rainbow_call_option_payoff(asset_prices, strike_price):
    return np.maximum(np.max(asset_prices, axis=1) - strike_price, 0)

def rainbow_value(current_asset_prices, in_the_money):
    return np.max(current_asset_prices[in_the_money, :], axis=1)

def rainbow_in_the_money(current_asset_prices, strike_price):
    return np.where(np.max(current_asset_prices, axis=1) > strike_price)[0]

### Worst-of Option

In [ ]:
def worst_of_put_option_payoff(asset_prices, strike_price):
    return np.maximum(strike_price - np.min(asset_prices, axis=1), 0)

def worst_value(current_asset_prices, in_the_money):
    return np.min(current_asset_prices[in_the_money, :], axis=1)

def worst_put_in_the_money(current_asset_prices, strike_price):
    return np.where(np.min(current_asset_prices, axis=1) < strike_price)[0]

### Spread Option

In [ ]:
# Spread Call Option (Max Asset - Min Asset)
def spread_call_option_payoff(asset_prices, strike_price):
    spread = np.max(asset_prices, axis=1) - np.min(asset_prices, axis=1)
    return np.maximum(spread - strike_price, 0)

def spread_value(current_asset_prices, in_the_money):
    return np.max(current_asset_prices[in_the_money, :], axis=1) - np.min(current_asset_prices[in_the_money, :], axis=1)

def spread_in_the_money(current_asset_prices, strike_price):
    spread = np.max(current_asset_prices, axis=1) - np.min(current_asset_prices, axis=1)
    return np.where(spread > strike_price)[0]

### Average Asset Option

In [ ]:
def average_asset_option_payoff(asset_prices, strike_price):
    return np.maximum(np.mean(asset_prices, axis=1) - strike_price, 0)

def average_asset_value(current_asset_prices, in_the_money):
    return np.mean(current_asset_prices[in_the_money, :], axis=1)

def average_asset_in_the_money(current_asset_prices, strike_price):
    return np.where(np.mean(current_asset_prices, axis=1) > strike_price)[0]

### 1D Call Option

In [ ]:
def payoff_call(S, K):
    return np.maximum(S[:, 0] - K, 0)

def price_sum(S, indices):
    return S[indices].sum(axis=1)

def in_the_money_call(S, K):
    return np.where(S[:, 0] > K)[0]

### 1D Put Option

In [ ]:
def payoff_put(S, K):
    return np.maximum(K - S[:, 0], 0)

def price_sum_put(S, indices):
    return S[indices, 0]

def in_the_money_put(S, K):
    return np.where(K > S[:, 0])[0]

### Strangle Spread

In [ ]:
def strangle_spread_payoff(asset_prices, strike_prices):
    K1, K2, K3, K4 = strike_prices
    average_asset_price = np.mean(asset_prices, axis=1)

    # Payoff from the long put spread (K2 - K1)
    put_spread_payoff = np.maximum(K2 - average_asset_price, 0) - np.maximum(K1 - average_asset_price, 0)

    # Payoff from the long call spread (K4 - K3)
    call_spread_payoff = np.maximum(average_asset_price - K3, 0) - np.maximum(average_asset_price - K4, 0)

    # Total payoff is the sum of the two spreads
    total_payoff = put_spread_payoff + call_spread_payoff
    return total_payoff

def strangle_spread_value(current_asset_prices, in_the_money, strike_prices):
    # Calculate the payoff for the in-the-money paths at the current time step using the average asset price
    return strangle_spread_payoff(current_asset_prices[in_the_money, :], strike_prices)

def strangle_spread_in_the_money(current_asset_prices, strike_prices):
    # A path is in the money if the immediate exercise value is greater than zero
    payoffs_at_current_time = strangle_spread_payoff(current_asset_prices, strike_prices)
    return np.where(payoffs_at_current_time > 0)[0]

### Min put

In [ ]:
def payoff_min_put(asset_prices, strike_price):
    return np.max(np.maximum(strike_price - asset_prices, 0), axis=1)

# 2. Asset Paths Plot

In [ ]:
##############################################################################################################################
initial_price     = np.array([60, 100])
strike_price      = 220
risk_free_rate    = 0.05
time_to_maturity  = 1
n_steps           = 1000
n_simulations     = 1000

volatilities = np.array([0.3, 0.2])

correlation_matrix = np.array([[1.0, 0.7],
                               [0.7, 1.0]])

n_assets = len(volatilities)
exercise_points = np.arange(n_steps)
asset_paths = simulate_asset_paths(n_simulations, n_steps, n_assets, time_to_maturity, risk_free_rate, volatilities, correlation_matrix, cholesky_cov, initial_price)
#############################################################################################################################plt.figure(figsize=(12, 6))

colors = plt.get_cmap('tab10').colors
for i in range(min(10, n_simulations)): # Plot up to 10 paths
    for j in range(n_assets):
        plt.plot(asset_paths[i, :, j], label=f'Asset {j+1} Simulation {i+1}', color= colors[i])
plt.xlabel('Time Steps')
plt.ylabel('Asset Price')
plt.grid(True)
plt.show()

# 3. Benchmark Methods

In [ ]:
# Standardised parameters
##############################################################################################################################
initial_price     = np.array([95])
strike_price      = 100
risk_free_rate    = 0.05
time_to_maturity  = 1
n_steps           = 20
n_simulations     = 2000

volatilities = np.array([0.2])
correlation_matrix = np.array([[1.0]])
n_assets = len(volatilities)
exercise_points = np.arange(n_steps)
#############################################################################################################################

## LSM

### Basis Function Sensitivity

In [ ]:
# Test different polynomial degrees
basis_degrees = np.array([1, 2, 3, 4, 5, 6, 7, 8])
num_runs_per_degree = 50
prices = []
degrees_recorded = []

for deg in basis_degrees:
    print(f"  Testing basis degree {deg}...")
    for run in range(num_runs_per_degree):
        asset_paths = simulate_asset_paths(n_simulations, n_steps, n_assets, time_to_maturity, risk_free_rate,
                                           volatilities, correlation_matrix, cholesky_cov, initial_price)

        ls_price = longstaff_schwartz_pricing(asset_paths, strike_price, risk_free_rate, time_to_maturity,
                                              exercise_points, payoff_call, price_sum, in_the_money_call, deg)
        prices.append(np.mean(ls_price))
        degrees_recorded.append(deg)

prices = np.array(prices)
degrees_recorded = np.array(degrees_recorded)

# Calculate the mean price for each basis degree
mean_prices = []
for deg in basis_degrees:
    prices_for_degree = prices[degrees_recorded == deg]
    mean_prices.append(np.mean(prices_for_degree))

mean_prices = np.array(mean_prices)

# --- Calculate and plot the trend line ---
valid_degrees = basis_degrees[~np.isnan(mean_prices)]
valid_mean_prices = mean_prices[~np.isnan(mean_prices)]

# Fit a polynomial trend line
trend_coeffs = np.polyfit(valid_degrees, valid_mean_prices, 2)
trend_line = np.polyval(trend_coeffs, basis_degrees)

# Plot results
plt.figure(figsize=(7,4))
plt.scatter(degrees_recorded, prices, alpha=0.5, label='Individual Runs')
plt.plot(basis_degrees, mean_prices, 'o', color='red', label='Mean Price')
plt.plot(basis_degrees, trend_line, linestyle='-', color='red', label=f'Mean Trend')
plt.xlabel('Polynomial Degree of the Basis function')
plt.ylabel('Option Price')
plt.xticks(basis_degrees)
plt.grid(True)
plt.legend()
plt.savefig("ls_basis_sensitivity.png", dpi=300, bbox_inches='tight')
plt.show()

### Convergence check

In [ ]:
results = []
n_simulations_list = list(range(10, 5000, 200))

for n_sim in n_simulations_list:
    asset_paths = simulate_asset_paths(n_sim, n_steps, n_assets, time_to_maturity, risk_free_rate, volatilities,
                                       correlation_matrix, cholesky_cov, initial_price)
    ls_price = longstaff_schwartz_pricing(asset_paths, strike_price, risk_free_rate, time_to_maturity,
                                          exercise_points, basket_call_option_payoff, basket_value, basket_in_the_money)
    results.append(np.mean(ls_price))

# Plot convergence of the estimated price
plt.figure(figsize=(8,5))
plt.plot(n_simulations_list, results, marker='o')
plt.xscale('log')
plt.xlabel('Number of Simulations')
plt.ylabel('Option Price')
plt.legend()
plt.grid(True)
plt.show()

### Confidence interval

In [ ]:
n_simulations = 2000
num_repeats = 200
mean_option_values = []

for i in range(num_repeats):
    asset_paths = simulate_asset_paths(n_simulations, n_steps, n_assets, time_to_maturity,
                                       risk_free_rate, volatilities, correlation_matrix, cholesky_cov, initial_price)
    ls_price = longstaff_schwartz_pricing(asset_paths, strike_price, risk_free_rate, time_to_maturity,
        exercise_points, payoff_call, price_sum, in_the_money_call, basis_degree=2)

    mean_option_values.append(np.mean(ls_price))

# Overall mean and standard error
overall_mean = np.mean(mean_option_values)
std_error = statistics.stdev(mean_option_values)

# 95% normal confidence interval
alpha = 0.05
z_critical = scipy.stats.norm.ppf(1 - alpha/2)

confidence_interval = (overall_mean - z_critical * std_error, overall_mean + z_critical * std_error)

print(f"\nOverall Mean LSM Price over {num_repeats} runs: {overall_mean:.4f}")
print(f"Standard Error of the Mean: {std_error:.4f}")
print(f"95% Confidence Interval (Normal): ({confidence_interval[0]:.4f}, {confidence_interval[1]:.4f})")

# Confidence plot
plt.figure(figsize=(10, 6))
plt.plot(mean_option_values, marker='o', linestyle='', label='Mean Option Price (per run)')
plt.axhline(overall_mean, color='r', linestyle='--', label=f'Overall Mean ({overall_mean:.4f})')
plt.axhspan(confidence_interval[0], confidence_interval[1], color='red', alpha=0.2, label='95% Normal Confidence Interval')
plt.xlabel('Run Number')
plt.ylabel('Option Price')
plt.grid(True)
plt.legend()
plt.show()

## KKT

### Validation Plot

In [ ]:
asset_paths = simulate_asset_paths(n_simulations, n_steps, n_assets, time_to_maturity, risk_free_rate, volatilities, correlation_matrix, cholesky_cov, initial_price)

KKT_pricing_with_validation(asset_paths, strike_price, risk_free_rate, time_to_maturity, exercise_points,
    payoff_call, price_sum, in_the_money_call, plot_validation = True)

### Convergence

In [ ]:
results = []
n_simulations_list = list(range(10, 5010, 200))

for n_sim in n_simulations_list:
    asset_paths = simulate_asset_paths(n_sim, n_steps, n_assets, time_to_maturity, risk_free_rate, volatilities, correlation_matrix, cholesky_cov, initial_price)

    kkt_option_values = KKT_pricing_with_validation(asset_paths, strike_price, risk_free_rate, time_to_maturity, exercise_points,
            basket_call_option_payoff, basket_value, basket_in_the_money)
    results.append(np.mean(kkt_option_values))

# Convergence plot
plt.figure(figsize=(8,5))
plt.plot(n_simulations_list, results, marker='o', label=f'{n_steps} time steps')
plt.xscale('log')
plt.xlabel('Number of Simulations')
plt.ylabel('Option Price')
plt.legend()
plt.grid(True)
plt.show()

### Confidence interval

In [ ]:
n_simulations = 2000
num_repeats = 200
mean_option_values = []

for i in range(num_repeats):
    asset_paths = simulate_asset_paths(n_simulations, n_steps, n_assets, time_to_maturity,
                                       risk_free_rate, volatilities, correlation_matrix, cholesky_cov, initial_price)
    kkt_option_values = KKT_pricing_with_validation(asset_paths, strike_price, risk_free_rate, time_to_maturity,
                                                    exercise_points, basket_call_option_payoff, basket_value, basket_in_the_money)

    mean_option_values.append(np.mean(kkt_option_values))
    print(f"Completed run {i+1}/{num_repeats}, Mean Price: {mean_option_values[-1]:.4f}")


# Overall mean and standard error
overall_mean = np.mean(mean_option_values)
std_error = statistics.stdev(mean_option_values)

# 95% normal confidence interval
alpha = 0.05
z_critical = scipy.stats.norm.ppf(1 - alpha/2)
confidence_interval = (overall_mean - z_critical * std_error, overall_mean + z_critical * std_error)

print(f"\nOverall Mean LSM Price over {num_repeats} runs: {overall_mean:.4f}")
print(f"Standard Error of the Mean: {std_error:.4f}")
print(f"95% Confidence Interval (Normal): ({confidence_interval[0]:.4f}, {confidence_interval[1]:.4f})")

# Confidence interval plot
plt.figure(figsize=(10, 6))
plt.plot(mean_option_values, marker='o', linestyle='', label='Mean KKT Price per Run')
plt.axhline(overall_mean, color='r', linestyle='--', label=f'Overall Mean ({overall_mean:.4f})')
plt.axhspan(confidence_interval[0], confidence_interval[1], color='red', alpha=0.2, label='95% Normal Confidence Interval')
plt.xlabel('Run Number')
plt.ylabel('Option Price')
plt.grid(True)
plt.legend()
plt.show()

## Dual

### Validation

In [ ]:
asset_paths = simulate_asset_paths(n_simulations, n_steps, n_assets, time_to_maturity, risk_free_rate, volatilities, correlation_matrix, cholesky_cov, initial_price)

deep_dual_pricing(asset_paths, strike_price, risk_free_rate, time_to_maturity, exercise_points,
    basket_call_option_payoff, num_epochs= 200, early_stopping_patience=50 ,plot_validation= True)

### Convergence

In [ ]:
results = []
n_simulations_list = list(range(10, 5010, 200))

for n_sim in n_simulations_list:
    asset_paths = simulate_asset_paths(n_sim, n_steps, n_assets, time_to_maturity, risk_free_rate, volatilities, correlation_matrix, cholesky_cov, initial_price)

    deep_dual_price = deep_dual_pricing(asset_paths, strike_price, risk_free_rate, time_to_maturity, exercise_points, basket_call_option_payoff)
    results.append(deep_dual_price)

# Convergence plot
plt.figure(figsize=(8,5))
plt.plot(n_simulations_list, results, marker='o', label=f'{n_steps} time steps')
plt.xscale('log')
plt.xlabel('Number of Simulations')
plt.ylabel('Option Price')
plt.legend()
plt.grid(True)
plt.show()

### Confidence Interval

In [ ]:
n_simulations = 2000
num_repeats = 200
results = []

for i in range(num_repeats):
    asset_paths = simulate_asset_paths(n_simulations, n_steps, n_assets, time_to_maturity,
                                       risk_free_rate, volatilities, correlation_matrix, cholesky_cov, initial_price)
    deep_dual_price = deep_dual_pricing(asset_paths, strike_price, risk_free_rate, time_to_maturity, exercise_points,
    basket_call_option_payoff, num_epochs=400, early_stopping_patience=60)
    results.append(np.min(deep_dual_price))
    print(f"Completed run {i+1}/{num_repeats}, Min Upper Bound: {results[-1]:.4f}")


# Overall mean and standard error
overall_mean = np.mean(results)
std_error = statistics.stdev(results)

# 95% normal confidence interval
alpha = 0.05
z_critical = scipy.stats.norm.ppf(1 - alpha/2)
confidence_interval = (overall_mean - z_critical * std_error, overall_mean + z_critical * std_error)

print(f"\nOverall Mean LSM Price over {num_repeats} runs: {overall_mean:.4f}")
print(f"Standard Error of the Mean: {std_error:.4f}")
print(f"95% Confidence Interval (Normal): ({confidence_interval[0]:.4f}, {confidence_interval[1]:.4f})")

# Confidence interval plot
plt.figure(figsize=(10, 6))
plt.plot(results, marker='o', linestyle='', label='Minimum upper bound per Run')
plt.axhline(overall_mean, color='r', linestyle='--', label=f'Overall Mean ({overall_mean:.4f})')
plt.axhspan(confidence_interval[0], confidence_interval[1], color='red', alpha=0.2, label='95% Normal Confidence Interval')
plt.xlabel('Run Number')
plt.ylabel('Option Price')
plt.grid(True)
plt.legend()
plt.show()

# 4. Comparisons

## Analytical Comparison

In [ ]:
##############################################################################################################################
# Parameters
strike_price      = 100
risk_free_rate    = 0.06
time_to_maturity  = 0.5
n_steps           = 1
n_simulations     = 2000

volatilities = np.array([0.4])
correlation_matrix = np.array([[1]])
n_assets = len(volatilities)
exercise_points = np.array([1])

initial_prices = np.array([80, 85, 90, 95, 100, 105, 110, 115, 120])
results = []
n_runs = 20
##############################################################################################################################

for initial_price in initial_prices:
    ls_runs = []
    kkt_runs = []
    deep_dual_runs = []

    for run in range(n_runs):
        asset_paths = simulate_asset_paths(
            n_simulations, n_steps, n_assets, time_to_maturity,
            risk_free_rate, volatilities, correlation_matrix,
            cholesky_cov, initial_price
        )

        # Longstaff-Schwartz
        ls_price_run = longstaff_schwartz_pricing(
            asset_paths, strike_price, risk_free_rate, time_to_maturity,
            exercise_points, payoff_put, price_sum_put, in_the_money_put
        )
        ls_runs.append(np.mean(ls_price_run))

        # KKT
        kkt_price_run = KKT_pricing_with_validation(
            asset_paths, strike_price, risk_free_rate, time_to_maturity,
            exercise_points, payoff_put, price_sum_put, in_the_money_put,
            early_stopping_patience=40
        )
        kkt_runs.append(np.mean(kkt_price_run))

        # Deep Dual
        deep_dual_price_run = deep_dual_pricing(
            asset_paths, strike_price, risk_free_rate, time_to_maturity,
            exercise_points, payoff_put, early_stopping_patience=80
        )
        deep_dual_runs.append(np.mean(deep_dual_price_run))

    # Means and std deviations
    ls_mean, ls_std = np.mean(ls_runs), np.std(ls_runs)
    kkt_mean, kkt_std = np.mean(kkt_runs), np.std(kkt_runs)
    deep_dual_mean, deep_dual_std = np.mean(deep_dual_runs), np.std(deep_dual_runs)

    # Black-Scholes benchmark
    S0 = initial_price
    K = strike_price
    r = risk_free_rate
    T = time_to_maturity
    sigma = volatilities[0]
    bs_price = black_scholes_put_price(S0, K, r, T, sigma)

    results.append({
        "Initial Price": S0,
        "LSM Mean": ls_mean,
        "LSM Std": ls_std,
        "KKT Mean": kkt_mean,
        "KKT Std": kkt_std,
        "Deep Dual Mean": deep_dual_mean,
        "Deep Dual Std": deep_dual_std,
        "Black-Scholes": bs_price
    })

results_df = pd.DataFrame(results)
pd.set_option("display.float_format", lambda x: f"{x:,.4f}")
print(results_df)

## Spread Option

In [ ]:
##############################################################################################################################
scenarios_spread_call = [
    {
        'name': 'Scenario 1 (Small Spread)',
        'initial_price': np.array([100, 105]), # Small spread
        'strike_price': 5,
        'risk_free_rate': 0.05,
        'time_to_maturity': 1.0,
        'volatilities': np.array([0.2, 0.2]),
        'correlation_matrix': np.array([[1.0, 0.5], [0.5, 1.0]])
    },
    {
        'name': 'Scenario 2 (Larger Spread)',
        'initial_price': np.array([90, 110]), # Larger spread
        'strike_price': 5,
        'risk_free_rate': 0.05,
        'time_to_maturity': 1.0,
        'volatilities': np.array([0.2, 0.2]),
        'correlation_matrix': np.array([[1.0, 0.5], [0.5, 1.0]])
    },
    {
        'name': 'Scenario 3 (Different Volatilities)',
        'initial_price': np.array([100, 100]),
        'strike_price': 5,
        'risk_free_rate': 0.05,
        'time_to_maturity': 1.0,
        'volatilities': np.array([0.1, 0.3]), # Different volatilities
        'correlation_matrix': np.array([[1.0, 0.5], [0.5, 1.0]])
    },
    {
        'name': 'Scenario 4 (Low Correlation)',
        'initial_price': np.array([100, 100]),
        'strike_price': 5,
        'risk_free_rate': 0.05,
        'time_to_maturity': 1.0,
        'volatilities': np.array([0.2, 0.2]),
        'correlation_matrix': np.array([[1.0, 0.1], [0.1, 1.0]]) # Low correlation (increases spread volatility)
    },
    {
        'name': 'Scenario 5 (Longer Maturity)',
        'initial_price': np.array([100, 105]),
        'strike_price': 5,
        'risk_free_rate': 0.05,
        'time_to_maturity': 2.0, # Longer maturity
        'volatilities': np.array([0.2, 0.2]),
        'correlation_matrix': np.array([[1.0, 0.5], [0.5, 1.0]])
    }
]
##############################################################################################################################################
# For all option types
n_steps_common = 5
n_simulations_common = 2000
num_simulation_runs = 20

option_payoff_func = spread_call_option_payoff
option_value_func = spread_value
option_in_the_money_func = spread_in_the_money

all_scenario_results_summary = []
print(f"--- Pricing Spread Option ---")

for i, scenario in enumerate(scenarios_spread_call):
    scenario_name = scenario['name']
    print(f"\nProcessing {scenario_name}...")

    initial_price = scenario['initial_price']
    strike_price = scenario['strike_price']
    risk_free_rate = scenario['risk_free_rate']
    time_to_maturity = scenario['time_to_maturity']
    volatilities = scenario['volatilities']
    correlation_matrix = scenario['correlation_matrix']
    n_assets = len(initial_price)
    exercise_points = np.arange(n_steps_common)

    ls_prices_scenario = []
    kkt_prices_scenario = []
    deep_dual_prices_scenario = []

    for run in range(num_simulation_runs):
        seed_value = i * num_simulation_runs + run
        np.random.seed(seed_value)
        tf.random.set_seed(seed_value)

        asset_paths = simulate_asset_paths(n_simulations_common, n_steps_common, n_assets, time_to_maturity, risk_free_rate, volatilities, correlation_matrix, cholesky_cov, initial_price)

        # Longstaff-Schwartz
        ls_price = longstaff_schwartz_pricing(asset_paths, strike_price, risk_free_rate, time_to_maturity,
                                              exercise_points, option_payoff_func, option_value_func, option_in_the_money_func)
        ls_prices_scenario.append(np.mean(ls_price))

        # KKT
        kkt_option_values = KKT_pricing_with_validation(
            asset_paths, strike_price, risk_free_rate, time_to_maturity, exercise_points,
            option_payoff_func, option_value_func, option_in_the_money_func
        )
        kkt_prices_scenario.append(np.mean(kkt_option_values))

        # Deep Dual
        deep_dual_price = deep_dual_pricing(
                asset_paths, strike_price, risk_free_rate, time_to_maturity,
                exercise_points,
                option_payoff_func
            )
        deep_dual_prices_scenario.append(deep_dual_price)

    scenario_results_summary = {
        'Scenario': scenario_name,
        'Option Type': 'Spread Call',
        'LSM Mean': np.nanmean(ls_prices_scenario),
        'LSM Std Dev': np.nanstd(ls_prices_scenario),
        'KKT Mean': np.nanmean(kkt_prices_scenario),
        'KKT Std Dev': np.nanstd(kkt_prices_scenario),
        'Deep Dual Mean': np.nanmean(deep_dual_prices_scenario),
        'Deep Dual Std Dev': np.nanstd(deep_dual_prices_scenario)
    }
    all_scenario_results_summary.append(scenario_results_summary)

results_df = pd.DataFrame(all_scenario_results_summary)
print("\nAggregated Comparison Results Table (Spread Call):")
display(results_df)

## Model efficiency

### vs assets

In [ ]:
##########################################################################################################
# Simulation Parameters
##########################################################################################################
n_assets_list = [1, 5, 50, 100, 500]
n_simulations = 2000
n_steps = 5
strike_price = 100
risk_free_rate = 0.05
time_to_maturity = 1.0
correlation_value = 0.3

n_runs = 20
results = []


##########################################################################################################
# Main Experiment Loop
##########################################################################################################
for n_asset in n_assets_list:
    print(f"\nRunning experiments for {n_asset} assets...")

    # Containers for multiple runs
    #lsm_prices, lsm_times = [], []
    #kkt_prices, kkt_times = [], []
    #deep_dual_prices, deep_dual_times = [], []

    for run in range(n_runs):
        initial_price = np.full(n_asset, 100.0)
        volatilities = np.full(n_asset, 0.2)

        correlation_matrix = np.full((n_asset, n_asset), correlation_value)
        np.fill_diagonal(correlation_matrix, 1.0)

        # Simulate correlated asset paths
        asset_paths = simulate_asset_paths(
            n_simulations, n_steps, n_asset, time_to_maturity,
            risk_free_rate, volatilities, correlation_matrix, cholesky_cov, initial_price
        )

        exercise_points = np.arange(n_steps)

        ###################################################################################################
        # LSM
        ###################################################################################################
        start_time = time.time()
        lsm_values = longstaff_schwartz_pricing(
            asset_paths, strike_price, risk_free_rate, time_to_maturity,
            exercise_points, average_asset_option_payoff, average_asset_value, average_asset_in_the_money
        )
        lsm_times.append(time.time() - start_time)
        lsm_prices.append(np.mean(lsm_values))

        ###################################################################################################
        # KKT
        ###################################################################################################
        start_time = time.time()
        kkt_values = KKT_pricing_with_validation(
            asset_paths, strike_price, risk_free_rate, time_to_maturity,
            exercise_points, average_asset_option_payoff, average_asset_value, average_asset_in_the_money
        )
        kkt_times.append(time.time() - start_time)
        kkt_prices.append(np.mean(kkt_values))


        ###################################################################################################
        # Deep Dual
        ###################################################################################################
        if n_asset <= 1:
            patience = 30
        else:
            patience = 100

        start_time = time.time()
        deep_dual_values = deep_dual_pricing(
            asset_paths, strike_price, risk_free_rate, time_to_maturity,
            exercise_points, average_asset_option_payoff, early_stopping_patience=patience
        )
        deep_dual_times.append(time.time() - start_time)
        deep_dual_prices.append(np.mean(deep_dual_values))

    ###################################################################################################
    # Results
    ###################################################################################################
    results.append({
        'Assets': n_asset,
        'LSM Price Mean': np.mean(lsm_prices), 'LSM Price Std': np.std(lsm_prices),
        'LSM Time Mean': np.mean(lsm_times),
        'KKT Price Mean': np.mean(kkt_prices), 'KKT Price Std': np.std(kkt_prices),
        'KKT Time Mean': np.mean(kkt_times),
        'Deep Dual Price Mean': np.mean(deep_dual_prices), 'Deep Dual Price Std': np.std(deep_dual_prices),
        'Deep Dual Time Mean': np.mean(deep_dual_times)
    })

##########################################################################################################
# Display Results
##########################################################################################################
df_results = pd.DataFrame(results)

print("\nRuntime and Pricing Results (Averaged over 5 runs)")
print("-" * 135)
print(f"{'Assets':<8} | {'LSM Mean':<10} | {'LSM Std':<10} | {'LSM Time':<10} | "
      f"{'KKT Mean':<10} | {'KKT Std':<10} | {'KKT Time':<10} | "
      f"{'Deep Dual Mean':<15} | {'Deep Dual Std':<12} | {'Deep Dual Time':<12}")
print("-" * 135)

for r in results:
    print(f"{r['Assets']:<8} | {r['LSM Price Mean']:<10.4f} | {r['LSM Price Std']:<10.4f} | {r['LSM Time Mean']:<10.4f} | "
          f"{r['KKT Price Mean']:<10.4f} | {r['KKT Price Std']:<10.4f} | {r['KKT Time Mean']:<10.4f} | "
          f"{r['Deep Dual Price Mean']:<15.4f} | {r['Deep Dual Price Std']:<12.4f} | {r['Deep Dual Time Mean']:<12.4f}")

### vs simulations

In [ ]:
##########################################################################################################
# Simulation Parameters
##########################################################################################################
n_simulations_list = [100, 500, 1000, 5000, 10000]
n_steps = 5
n_assets = 3
strike_price = 100
risk_free_rate = 0.05
time_to_maturity = 1.0
correlation_value = 0.3

n_runs = 20
results = []

##########################################################################################################
# Main Experiment Loop
##########################################################################################################
for n_simulations in n_simulations_list:
    print(f"\nRunning experiments for {n_simulations} simulations...")

    lsm_prices, lsm_times = [], []
    kkt_prices, kkt_times = [], []
    deep_dual_prices, deep_dual_times = [], []

    for run in range(n_runs):
        initial_price = np.full(n_assets, 100.0)
        volatilities = np.full(n_assets, 0.2)

        correlation_matrix = np.full((n_assets, n_assets), correlation_value)
        np.fill_diagonal(correlation_matrix, 1.0)

        asset_paths = simulate_asset_paths(
            n_simulations, n_steps, n_assets, time_to_maturity,
            risk_free_rate, volatilities, correlation_matrix, cholesky_cov, initial_price
        )

        exercise_points = np.arange(n_steps)

        ###################################################################################################
        # LSM
        ###################################################################################################
        start_time = time.time()
        lsm_values = longstaff_schwartz_pricing(
            asset_paths, strike_price, risk_free_rate, time_to_maturity,
            exercise_points, rainbow_call_option_payoff, rainbow_value, rainbow_in_the_money
        )
        lsm_times.append(time.time() - start_time)
        lsm_prices.append(np.mean(lsm_values))

        ###################################################################################################
        # KKT
        ###################################################################################################
        start_time = time.time()
        kkt_values = KKT_pricing_with_validation(
            asset_paths, strike_price, risk_free_rate, time_to_maturity,
            exercise_points, rainbow_call_option_payoff, rainbow_value, rainbow_in_the_money
        )
        kkt_times.append(time.time() - start_time)
        kkt_prices.append(np.mean(kkt_values))

        ###################################################################################################
        # Deep Dual
        ###################################################################################################
        start_time = time.time()
        deep_dual_values = deep_dual_pricing(
            asset_paths, strike_price, risk_free_rate, time_to_maturity,
            exercise_points, rainbow_call_option_payoff, early_stopping_patience=80
        )
        deep_dual_times.append(time.time() - start_time)
        deep_dual_prices.append(np.mean(deep_dual_values))

    ###################################################################################################
    # Results
    ###################################################################################################
    results.append({
        'Simulations': n_simulations,
        'LSM Price Mean': np.mean(lsm_prices), 'LSM Price Std': np.std(lsm_prices),
        'LSM Time Mean': np.mean(lsm_times),
        'KKT Price Mean': np.mean(kkt_prices), 'KKT Price Std': np.std(kkt_prices),
        'KKT Time Mean': np.mean(kkt_times),
        'Deep Dual Price Mean': np.mean(deep_dual_prices), 'Deep Dual Price Std': np.std(deep_dual_prices),
        'Deep Dual Time Mean': np.mean(deep_dual_times)
    })

##########################################################################################################
# Display Results
##########################################################################################################
df_results = pd.DataFrame(results)

print("\nRuntime and Pricing Results (Averaged over 5 runs)")
print("-" * 140)
print(f"{'Simulations':<12} | {'LSM Mean':<10} | {'LSM Std':<10} | {'LSM Time':<10} | "
      f"{'KKT Mean':<10} | {'KKT Std':<10} | {'KKT Time':<10} | "
      f"{'Deep Dual Mean':<15} | {'Deep Dual Std':<12} | {'Deep Dual Time':<12}")
print("-" * 140)

for r in results:
    print(f"{r['Simulations']:<12} | {r['LSM Price Mean']:<10.4f} | {r['LSM Price Std']:<10.4f} | {r['LSM Time Mean']:<10.4f} | "
          f"{r['KKT Price Mean']:<10.4f} | {r['KKT Price Std']:<10.4f} | {r['KKT Time Mean']:<10.4f} | "
          f"{r['Deep Dual Price Mean']:<15.4f} | {r['Deep Dual Price Std']:<12.4f} | {r['Deep Dual Time Mean']:<12.4f}")


### vs steps

In [ ]:
##########################################################################################################
# Simulation Parameters
##########################################################################################################
n_steps_list = [1, 5, 10, 50, 100]
n_simulations = 2000
n_assets = 3
strike_price = 100
risk_free_rate = 0.05
time_to_maturity = 1.0
correlation_value = 0.3

n_runs = 20
results = []

##########################################################################################################
# Main Experiment Loop
##########################################################################################################
for n_steps in n_steps_list:
    print(f"\nRunning experiments for {n_steps} time steps...")

    lsm_prices, lsm_times = [], []
    kkt_prices, kkt_times = [], []
    deep_dual_prices, deep_dual_times = [], []

    for run in range(n_runs):
        initial_price = np.full(n_assets, 100.0)
        volatilities = np.full(n_assets, 0.2)

        correlation_matrix = np.full((n_assets, n_assets), correlation_value)
        np.fill_diagonal(correlation_matrix, 1.0)

        asset_paths = simulate_asset_paths(
            n_simulations, n_steps, n_assets, time_to_maturity,
            risk_free_rate, volatilities, correlation_matrix, cholesky_cov, initial_price
        )

        exercise_points = np.arange(n_steps)

        ###################################################################################################
        # LSM
        ###################################################################################################
        start_time = time.time()
        lsm_values = longstaff_schwartz_pricing(
            asset_paths, strike_price, risk_free_rate, time_to_maturity,
            exercise_points, rainbow_call_option_payoff, rainbow_value, rainbow_in_the_money
        )
        lsm_times.append(time.time() - start_time)
        lsm_prices.append(np.mean(lsm_values))

        ###################################################################################################
        # KKT
        ###################################################################################################
        start_time = time.time()
        kkt_values = KKT_pricing_with_validation(
            asset_paths, strike_price, risk_free_rate, time_to_maturity,
            exercise_points, rainbow_call_option_payoff, rainbow_value, rainbow_in_the_money
        )
        kkt_times.append(time.time() - start_time)
        kkt_prices.append(np.mean(kkt_values))

        ###################################################################################################
        # Deep Dual
        ###################################################################################################
        start_time = time.time()
        deep_dual_values = deep_dual_pricing(
            asset_paths, strike_price, risk_free_rate, time_to_maturity,
            exercise_points, rainbow_call_option_payoff, early_stopping_patience=80
        )
        deep_dual_times.append(time.time() - start_time)
        deep_dual_prices.append(np.mean(deep_dual_values))

    ###################################################################################################
    # Results
    ###################################################################################################
    results.append({
        'Time Steps': n_steps,
        'LSM Price Mean': np.mean(lsm_prices), 'LSM Price Std': np.std(lsm_prices),
        'LSM Time Mean': np.mean(lsm_times),
        'KKT Price Mean': np.mean(kkt_prices), 'KKT Price Std': np.std(kkt_prices),
        'KKT Time Mean': np.mean(kkt_times),
        'Deep Dual Price Mean': np.mean(deep_dual_prices), 'Deep Dual Price Std': np.std(deep_dual_prices),
        'Deep Dual Time Mean': np.mean(deep_dual_times)
    })

##########################################################################################################
# Display Results
##########################################################################################################
df_results = pd.DataFrame(results)

print("\nRuntime and Pricing Results (Averaged over 5 runs)")
print("-" * 140)
print(f"{'Steps':<8} | {'LSM Mean':<10} | {'LSM Std':<10} | {'LSM Time':<10} | "
      f"{'KKT Mean':<10} | {'KKT Std':<10} | {'KKT Time':<10} | "
      f"{'Deep Dual Mean':<15} | {'Deep Dual Std':<12} | {'Deep Dual Time':<12}")
print("-" * 140)

for r in results:
    print(f"{r['Time Steps']:<8} | {r['LSM Price Mean']:<10.4f} | {r['LSM Price Std']:<10.4f} | {r['LSM Time Mean']:<10.4f} | "
          f"{r['KKT Price Mean']:<10.4f} | {r['KKT Price Std']:<10.4f} | {r['KKT Time Mean']:<10.4f} | "
          f"{r['Deep Dual Price Mean']:<15.4f} | {r['Deep Dual Price Std']:<12.4f} | {r['Deep Dual Time Mean']:<12.4f}")


## Papers

### KKT

In [ ]:
# Single-Asset Strangle Spread
################################################################################################################################
initial_price     = np.array([100])
strike_price      = np.array([50, 90, 110, 150])
risk_free_rate    = 0.05
time_to_maturity  = 1
n_steps           = 12
n_simulations     = 2000

volatilities = np.array([0.25])
correlation_matrix = np.array([[1]])
n_assets = len(volatilities)
exercise_points = np.arange(n_steps)
################################################################################################################################

num_repeats = 20
ls_prices = []
kkt_prices = []

print(f"Running comparisons for {num_repeats} repeats with {n_simulations} simulations each...")

for i in range(num_repeats):
    print(f"Repeat {i+1}/{num_repeats}")

    asset_paths = simulate_asset_paths(
        n_simulations, n_steps, n_assets, time_to_maturity,
        risk_free_rate, volatilities, correlation_matrix,
        cholesky_cov, initial_price
    )

    # Longstaff-Schwartz
    ls_price_run = longstaff_schwartz_pricing(
        asset_paths, strike_price, risk_free_rate, time_to_maturity,
        exercise_points, strangle_spread_payoff,
        strangle_spread_value, strangle_spread_in_the_money,
        basis_degree=3
    )
    ls_prices.append(np.mean(ls_price_run))

    # KKT
    kkt_price_run = KKT_pricing_with_validation(
        asset_paths, strike_price, risk_free_rate, time_to_maturity,
        exercise_points, strangle_spread_payoff,
        strangle_spread_value, strangle_spread_in_the_money
    )
    kkt_prices.append(np.mean(kkt_price_run))

################################################################################################################################
# Quantiles from KKT paper boxplots (for comparison)
custom_TR  = [11.15, 11.30, 11.45, 11.55, 11.70]
custom_LS  = [11.05, 11.30, 11.45, 11.55, 11.75]
custom_KKT = [11.10, 11.35, 11.50, 11.65, 11.80]
################################################################################################################################

# Display in table
results_strangle = {
    "LSM (Our Model)" : np.mean(ls_prices),
    "KKT (Our Model)" : np.mean(kkt_prices),
    "LSM"             : np.mean(custom_LS),
    "KKT"             : np.mean(custom_KKT),
    "TR"              : np.mean(custom_TR)
}

print("\n--- Table 15: Single-Asset Strangle Spread ---")
print(pd.DataFrame([results_strangle]))
################################################################################################################################

# Boxplot comparison
data_to_plot = [ls_prices, custom_LS, kkt_prices, custom_KKT, custom_TR]
labels = ['LSM (Our Model)', 'LSM', 'KKT (Our Model)', 'KKT', 'TR']

plt.figure(figsize=(10, 6))
plt.boxplot(data_to_plot, labels=labels, showfliers=True)
plt.ylabel('Option Price')
plt.grid(axis='y')
plt.show()

### Bekker


In [ ]:
# Single-asset Bermudan Put
##############################################################################################################################
def run_bermudan(d, n_simulations=2000, num_repeats=20):
    r, beta, K, chi = 0.02, 0.3, 90, 95
    T, N = 1, 2
    n_steps = N
    initial_price = np.ones(d) * chi
    volatilities = np.ones(d) * beta
    correlation_matrix = np.full((d, d), 0.1) + np.eye(d) * 0.9
    exercise_points = np.arange(n_steps)

    ls_prices, kkt_prices, dual_prices = [], [], []
    for k in range(num_repeats):
        print(k)
        asset_paths = simulate_asset_paths(
            n_simulations, n_steps, d,
            T, r, volatilities,
            correlation_matrix, cholesky_cov, initial_price
        )
        # LSM
        ls_price = longstaff_schwartz_pricing(
            asset_paths, K, r, T, exercise_points,
            payoff_put, price_sum_put, in_the_money_put
        )
        ls_prices.append(np.mean(ls_price))
        # KKT
        kkt_val = KKT_pricing_with_validation(
            asset_paths, K, r, T, exercise_points,
            payoff_put, price_sum_put, in_the_money_put
        )
        kkt_prices.append(np.mean(kkt_val))

    return {
        "LSM (Ours)": np.mean(ls_prices),
        "KKT (Ours)": np.mean(kkt_prices),
    }

# Collect results for Bermuda
dims_bermudan = [1, 5, 10, 50, 100, 500,1000]
results_bermudan = {d: run_bermudan(d) for d in dims_bermudan}

paper_bermudan = pd.DataFrame({
    "Mean of BCJW": [7.896, 7.893, 7.895, 7.889, 7.890, 7.894, 7.892],
}, index=dims_bermudan)

df_bermudan = pd.DataFrame(results_bermudan).T
comparison_bermudan = pd.concat([df_bermudan, paper_bermudan], axis=1)

print("\nBermudan Put — Our vs Paper:")
print(comparison_bermudan.round(3))

In [ ]:
# Multi-Asset Strangle Spread Comparison
################################################################################################################################
initial_price     = np.ones(5) * 100
strike_price      = np.array([75, 90, 110, 125])
risk_free_rate    = 0.05
time_to_maturity  = 1
n_steps           = 48
n_simulations     = 2000

covariance_matrix = np.array([
    [0.3024, 0.1354, 0.0722, 0.1367, 0.1641],
    [0.1354, 0.2270, 0.0613, 0.1264, 0.1610],
    [0.0722, 0.0613, 0.0717, 0.0884, 0.0699],
    [0.1367, 0.1264, 0.0884, 0.2937, 0.1394],
    [0.1641, 0.1610, 0.0699, 0.1394, 0.2535]
])

volatilities       = np.sqrt(np.diag(covariance_matrix))
correlation_matrix = covariance_matrix / np.outer(volatilities, volatilities)
n_assets           = len(volatilities)
exercise_points    = np.arange(n_steps)
################################################################################################################################

num_repeats = 20
ls_prices = []
kkt_prices = []

print(f"Running comparisons for {num_repeats} repeats with {n_simulations} simulations each...")

for i in range(num_repeats):
    print(f"Repeat {i+1}/{num_repeats}")

    asset_paths = simulate_asset_paths(
        n_simulations, n_steps, n_assets, time_to_maturity,
        risk_free_rate, volatilities, correlation_matrix,
        cholesky_cov, initial_price
    )

    # Longstaff-Schwartz
    ls_price_run = longstaff_schwartz_pricing(
        asset_paths, strike_price, risk_free_rate, time_to_maturity,
        exercise_points, payoff_strangle_spread,
        price_sum_strangle, in_the_money_strangle,
        basis_degree=3
    )
    ls_prices.append(np.mean(ls_price_run))

    # KKT
    kkt_price_run = KKT_pricing_with_validation(
        asset_paths, strike_price, risk_free_rate, time_to_maturity,
        exercise_points, payoff_strangle_spread,
        price_sum_strangle, in_the_money_strangle
    )
    kkt_prices.append(np.mean(kkt_price_run))

################################################################################################################################
# Results
################################################################################################################################
paper_value = 4.45

# Display in table
results_strangle = {
    "LSM (Our Model)" : np.mean(ls_prices),
    "KKT (Our Model)" : np.mean(kkt_prices),
    "Paper"           : paper_value
}

print("\n--- Table 14: Multi-Asset Strangle Spread ---")
print(pd.DataFrame([results_strangle]))

### Rogers

In [ ]:
##############################################################################################################################
# Parameters
##############################################################################################################################
strike_price      = 100
risk_free_rate    = 0.06
time_to_maturity  = 0.5
n_steps           = 50
n_simulations     = 10000

volatilities = np.array([0.4, 0.8])
correlation_matrix = np.array([[1, 0], [0, 1]])
n_assets = len(volatilities)
exercise_points = np.arange(n_steps)

initial_prices = np.array([[80,80], [80, 100], [80, 120], [100, 100], [100, 120], [120, 120]])
num_repeats = 20
results = []
##############################################################################################################################

for i in range(len(initial_prices)):
    deep_dual_prices = []
    for j in range(num_repeats):
        print(j)
        initial_price = initial_prices[i]
        asset_paths = simulate_asset_paths(
            n_simulations, n_steps, n_assets, time_to_maturity,
            risk_free_rate, volatilities, correlation_matrix, cholesky_cov, initial_price
        )

        # Deep Dual Pricing
        deep_dual_price_run = deep_dual_pricing(
            asset_paths, strike_price, risk_free_rate, time_to_maturity,
            exercise_points, payoff_min_put, early_stopping_patience=100, num_epochs= 700,
            hidden_dim=256, learning_rate=0.01,
        )
        deep_dual_prices.append(deep_dual_price_run)


    # Compute mean and standard deviation for this initial price pair
    print(deep_dual_prices)
    mean_price = np.mean(deep_dual_prices)
    std_price = np.std(deep_dual_prices)

    results.append({
        'Initial Price 1': initial_prices[i, 0],
        'Initial Price 2': initial_prices[i, 1],
        'Mean Deep Dual': mean_price,
        'Std Deep Dual': std_price
    })

##############################################################################################################################
# Display Results
##############################################################################################################################
df_results = pd.DataFrame(results)
print("\nDeep Dual Pricing Results (Averaged over repeats)")
print("-" * 70)
print(f"{'Initial (S1,S2)':<20} | {'Mean':<15} | {'Std Dev':<15}")
print("-" * 70)

for r in results:
    print(f"({r['Initial Price 1']:.0f}, {r['Initial Price 2']:.0f}){'':<6} | {r['Mean Deep Dual']:<15.4f} | {r['Std Deep Dual']:<15.4f}")

print("-" * 70)

# Optional: display neatly using pandas
display(df_results)

